In [1]:
from openfermion import QubitOperator, chemist_ordered
from numpy import imag
from copy import deepcopy
from utils import test_mapping, remap_modes


In [2]:
longstab = ({
    (0,1): QubitOperator('Z0 X24'),                             # 25
    (1,6): QubitOperator('Y0 Z1 Z2 Z3 Z4 X5 X12 Z24'),          # 13
    (6,7): QubitOperator('Y5 X6 Z12 X13'),                      # 14
    (7,12): QubitOperator('Y6 Z7 Z8 Z9 Z10 X11 Z13 X14'),       # 15
    (11,12): QubitOperator('Z10 Z11 Z14 X15'),                  # 16
    (8,11): QubitOperator('Y7 Z8 Z9 X10 Z15 X16 Z23'),          # 17
    (5,8): QubitOperator(' Y4 Z5 Z6 X7 Z16 X17'),               # 18
    (2,5): QubitOperator('Y1 Z2 Z3 X4 Z17 X18'),                # 19
    (2,3): QubitOperator('Z1 Z2 Z18 X19'),                      # 20
    (3,4): QubitOperator('Y2 X3 Z19 X20'),                      # 21
    (4,9): QubitOperator('Y3 Z4 Z5 Z6 Z7 X8 Z20 X21'),          # 22
    (9,10): QubitOperator('Y8 X9 Z21 X22'),                     # 23
    (10,11): QubitOperator('Z9 Z10 X15 Z16 Y22 Y23'),           # 24
})

corr = ({
    (0,1): QubitOperator('Z24'),                                # 25
    (1,6): QubitOperator('Z12'),                                # 13
    (6,7): QubitOperator('Z13'),                                # 14
    (7,12): QubitOperator('Z14'),                               # 15
    (11,12): QubitOperator('Z15 Z23'),                          # 16
    (8,11): QubitOperator('Z16'),                               # 17
    (5,8): QubitOperator('Z17'),                                # 18
    (2,5): QubitOperator('Z18'),                                # 19
    (2,3): QubitOperator('Z19'),                                # 20
    (3,4): QubitOperator('Z20'),                                # 21
    (4,9): QubitOperator('Z21'),                                # 22
    (9,10): QubitOperator('Z22 Z23'),                           # 23
    (10,11):  QubitOperator('Z23'),                             # 24
})

def anticoms(a,b, thresh=10**-8):
    """check if a and b anticommute"""
    return abs(imag(list((a*b).terms.values())[0])) >= thresh


def make_A(index_one, index_two):
    pstring = ((index_one-1, 'Y'),)
    
    for x in range(index_one, index_two-1):
        pstring += ((x, 'Z'),)
    
    pstring += ((index_two-1, 'X'),)
    return QubitOperator(pstring)


def correct_op(op):
    correction = QubitOperator(())
    for key, val in list(longstab.items()):
        if anticoms(op, val):
            correction *= corr[key]
    return op * correction


A_op = ({
    (1,6): correct_op(make_A(1,6)) * longstab[(1,6)],
    (6,7): correct_op(make_A(6,7)) * longstab[(6,7)],
    (7,12): correct_op(make_A(7,12)) * longstab[(7,12)],
    (8,11): correct_op(make_A(8,11)) * longstab[(8,11)],
    (5,8): correct_op(make_A(5,8)) * longstab[(5,8)],
    (2,5): correct_op(make_A(2,5)),
    (2,3): correct_op(make_A(2,3)) * longstab[(2,3)],
    (3,4): correct_op(make_A(3,4)) * longstab[(3,4)],
    (4,9): correct_op(make_A(4,9)) * longstab[(4,9)],
    (9,10): correct_op(make_A(9,10)) * longstab[(9,10)],
    (10,11): correct_op(make_A(10,11)) * longstab[(10,11)],
    (1,2): correct_op(make_A(1,2)),
    (2,3): correct_op(make_A(2,3)),
    (4,5): correct_op(make_A(4,5)),
    (5,6): correct_op(make_A(5,6)),
    (7,8): correct_op(make_A(7,8)),
    (8,9): correct_op(make_A(8,9)),
    (10,11): correct_op(make_A(10,11)),
    (11,12): correct_op(make_A(11,12)),
})

B_op = ({
    1: correct_op(QubitOperator('Z0')),
    2: correct_op(QubitOperator('Z1')),
    3: correct_op(QubitOperator('Z2')),
    4: correct_op(QubitOperator('Z3')),
    5: correct_op(QubitOperator('Z4')),
    6: correct_op(QubitOperator('Z5')),
    7: correct_op(QubitOperator('Z6')),
    8: correct_op(QubitOperator('Z7')),
    9: correct_op(QubitOperator('Z8')),
    10: correct_op(QubitOperator('Z9')),
    11: correct_op(QubitOperator('Z10')),
    12: correct_op(QubitOperator('Z11')),
    
})

stabs = [
    longstab[(1,6)],                        # 0
    longstab[(2,5)],                        # 1
    longstab[(3,4)],                        # 2
    longstab[(2,3)],                        # 3
    longstab[(4,9)],                        # 4
    longstab[(5,8)],                        # 5
    longstab[(6,7)],                        # 6
    longstab[(7,12)],                       # 7
    longstab[(8,11)],                       # 8
    longstab[(9,10)],                       # 9
    longstab[(10,11)],                      # 10
    longstab[(11,12)],                      # 11
    longstab[(0,1)],                        # 12

]

In [3]:
A_op

{(1, 6): 1.0 [X12 Z13],
 (6, 7): 1.0 [X13 Z14],
 (7, 12): 1.0 [X14 Z15 Z23],
 (8, 11): 1.0 [X16 Z17 Z23],
 (5, 8): 1.0 [X17 Z18],
 (2, 5): 1.0 [Y1 Z2 Z3 X4 Z17 Z19],
 (2, 3): 1.0 [Y1 X2 Z18 Z20],
 (3, 4): 1.0 [X20 Z21],
 (4, 9): 1.0 [X21 Z22 Z23],
 (9, 10): 1.0 [X22 Z23],
 (10, 11): 1.0 [Y9 X10 Z15 Z16 Z22],
 (1, 2): 1.0 [Y0 X1 Z12 Z18 Z19 Z24],
 (4, 5): 1.0 [Y3 X4 Z17 Z18 Z20 Z21],
 (5, 6): 1.0 [Y4 X5 Z12 Z13 Z17 Z18],
 (7, 8): 1.0 [Y6 X7 Z13 Z14 Z16 Z17],
 (8, 9): 1.0 [Y7 X8 Z16 Z17 Z21 Z22 Z23],
 (11, 12): 1.0 [Y10 X11 Z14 Z16 Z23]}

In [4]:
B_op

{1: 1.0 [Z0 Z12],
 2: 1.0 [Z1 Z18],
 3: 1.0 [Z2 Z20],
 4: 1.0 [Z3 Z20 Z21],
 5: 1.0 [Z4 Z17 Z18],
 6: 1.0 [Z5 Z12 Z13],
 7: 1.0 [Z6 Z13 Z14],
 8: 1.0 [Z7 Z16 Z17],
 9: 1.0 [Z8 Z21 Z22 Z23],
 10: 1.0 [Z9 Z22 Z23],
 11: 1.0 [Z10 Z16],
 12: 1.0 [Z11 Z14]}

In [5]:
table = {
    1: 0,
    2: 2,
    3: 4,
    4: 5,
    5: 3,
    6: 1,
    7: 6,
    8: 8,
    9: 10,
    10: 11,
    11: 9,
    12: 7
}

In [6]:
test_mapping(
        A_op,
        B_op,
        stabs)

No irregularities found.


In [7]:
new_A, new_B = remap_modes(table, (A_op, B_op))

In [8]:
new_A

{(0, 1): 1.0 [X12 Z13],
 (1, 6): 1.0 [X13 Z14],
 (6, 7): 1.0 [X14 Z15 Z23],
 (8, 9): 1.0 [X16 Z17 Z23],
 (3, 8): 1.0 [X17 Z18],
 (2, 3): 1.0 [Y1 Z2 Z3 X4 Z17 Z19],
 (2, 4): 1.0 [Y1 X2 Z18 Z20],
 (4, 5): 1.0 [X20 Z21],
 (5, 10): 1.0 [X21 Z22 Z23],
 (10, 11): 1.0 [X22 Z23],
 (9, 11): -1.0 [Y9 X10 Z15 Z16 Z22],
 (0, 2): 1.0 [Y0 X1 Z12 Z18 Z19 Z24],
 (3, 5): -1.0 [Y3 X4 Z17 Z18 Z20 Z21],
 (1, 3): -1.0 [Y4 X5 Z12 Z13 Z17 Z18],
 (6, 8): 1.0 [Y6 X7 Z13 Z14 Z16 Z17],
 (8, 10): 1.0 [Y7 X8 Z16 Z17 Z21 Z22 Z23],
 (7, 9): -1.0 [Y10 X11 Z14 Z16 Z23]}

In [9]:
new_B

{0: 1.0 [Z0 Z12],
 2: 1.0 [Z1 Z18],
 4: 1.0 [Z2 Z20],
 5: 1.0 [Z3 Z20 Z21],
 3: 1.0 [Z4 Z17 Z18],
 1: 1.0 [Z5 Z12 Z13],
 6: 1.0 [Z6 Z13 Z14],
 8: 1.0 [Z7 Z16 Z17],
 10: 1.0 [Z8 Z21 Z22 Z23],
 11: 1.0 [Z9 Z22 Z23],
 9: 1.0 [Z10 Z16],
 7: 1.0 [Z11 Z14]}

In [10]:
stabs= [

(1-0j) * QubitOperator('Z0 X3 Z4 X5 Z8'),
(-1+0j) * QubitOperator('Y0 X1 Z2 Y3 Z4 X5 Z9 X10 X11'),
(-1+0j) * QubitOperator('Y0 Y1 Z2 Z3 Y4 Y5 Z13'),
(-1+0j) * QubitOperator('X0 Z1 Y2 Z3 X4 Y5 X15 X16 Z17'),
(1+0j) * QubitOperator('X0 Z1 X2 Z5 Z18'),
(1-0j) * QubitOperator('X0 X1 Z2 X6 Z7 Y8 Z9 X10 Y11'),
(-1+0j) * QubitOperator('X0 Y1 Z2 Y6 Z7 Y8 Z12 Y13 X14'),
(-1+0j) * QubitOperator('Y0 Z1 Y2 Z6 Y7 X8 X15 Y16 Z17'),
(1+0j)  * QubitOperator('Y0 Z1 X2 Z6 X7 X8 Y18 X19 Z20'),
(1-0j) * QubitOperator('Z1 Y9 Y10 Z11 Z12 Y13 Y14'),
(-1+0j) * QubitOperator('Z0 Y1 X2 X9 Y10 Z11 Y15 Z16 Y17'),
(1+0j) * QubitOperator('Z0 Y1 Y2 Z10 Y18 Y19 Z20'),
(1-0j) * QubitOperator('Z0 X1 X2 Y12 X13 Z14 Y15 Z16 X17'),
(1-0j) * QubitOperator('Z0 X1 Y2 X12 X13 Z14 X18 Z19 Y20'),
(-1+0j) * QubitOperator('Z2 Z15 X18 Z19 X20'),
(1-0j) * QubitOperator('Z21 X24 Z25 X26 Z29'),
(-1+0j) * QubitOperator('Y21 X22 Z23 Y24 Z25 X26 Z30 X31 X32'),
(-1+0j) * QubitOperator('Y21 Y22 Z23 Z24 Y25 Y26 Z34'),
(-1+0j) * QubitOperator('X21 Z22 Y23 Z24 X25 Y26 X36 X37 Z38'),
(1+0j) * QubitOperator('X21 Z22 X23 Z26 Z39'),
(1-0j) * QubitOperator('X21 X22 Z23 X27 Z28 Y29 Z30 X31 Y32'),
(-1+0j) * QubitOperator('X21 Y22 Z23 Y27 Z28 Y29 Z33 Y34 X35'),
(-1+0j) * QubitOperator('Y21 Z22 Y23 Z27 Y28 X29 X36 Y37 Z38'),
(1+0j) * QubitOperator('Y21 Z22 X23 Z27 X28 X29 Y39 X40 Z41'),
(1-0j) * QubitOperator('Z22 Y30 Y31 Z32 Z33 Y34 Y35'),
(-1+0j) * QubitOperator('Z21 Y22 X23 X30 Y31 Z32 Y36 Z37 Y38'),
(1+0j) * QubitOperator('Z21 Y22 Y23 Z31 Y39 Y40 Z41'),
(1-0j) * QubitOperator('Z21 X22 X23 Y33 X34 Z35 Y36 Z37 X38'),
(1-0j) * QubitOperator('Z21 X22 Y23 X33 X34 Z35 X39 Z40 Y41'),
(-1+0j) * QubitOperator('Z23 Z36 X39 Z40 X41')]

In [11]:
bops = {
    0: (1+0j) * QubitOperator('Z0 Z1 Z2'),
    1: (1+0j) * QubitOperator('Z3 Z4 Z5'),
    2: (1+0j) * QubitOperator('Z6 Z7 Z8'),
    3: (1+0j) * QubitOperator('Z9 Z10 Z11'),
    4: (1+0j) * QubitOperator('Z12 Z13 Z14'),
    5: (1+0j) * QubitOperator('Z15 Z16 Z17'),
    6: (1+0j) * QubitOperator('Z18 Z19 Z20'),
    7: (1+0j) * QubitOperator('Z21 Z22 Z23'),
    8: (1+0j) * QubitOperator('Z24 Z25 Z26'),
    9: (1+0j) * QubitOperator('Z27 Z28 Z29'),
    10: (1+0j) * QubitOperator('Z30 Z31 Z32'),
    11: (1+0j) * QubitOperator('Z33 Z34 Z35'),
    12: (1+0j) * QubitOperator('Z36 Z37 Z38'),
    13: (1+0j) * QubitOperator('Z39 Z40 Z41'),
}

In [12]:
aops = {
    (0, 1): -1.0 * QubitOperator('X0 Z2 Z4 Y5'),
    (0, 2): 1.0 * QubitOperator('Y0 Z2 Z7 X8'),
    (0, 3): 1.0 * QubitOperator('Z0 X1 Z9 Y10'),
    (0, 4): 1.0 * QubitOperator('Z0 Y1 Z12 X13'),
    (0, 5): 1.0 * QubitOperator('Z1 X2 Y15 Z17'),
    (0, 6): 1.0 * QubitOperator('Z1 Y2 X18 Z20'),
    (1, 2): 1.0 * QubitOperator('X3 Z5 Z7 Y8'),
    (1, 3): 1.0 * QubitOperator('Y3 Z5 Z10 X11'),
    (1, 4): 1.0 * QubitOperator('Z3 X4 Z12 Y13'),
    (1, 5): 1.0 * QubitOperator('Z3 Y4 Z15 X16'),
    (1, 6): 1.0 * QubitOperator('Z4 X5 Y18 Z20'),
    (2, 3): 1.0 * QubitOperator('X6 Z8 Z10 Y11'),
    (2, 4): 1.0 * QubitOperator('Y6 Z8 Z13 X14'),
    (2, 5): 1.0 * QubitOperator('Z6 X7 Z15 Y16'),
    (2, 6): 1.0 * QubitOperator('Z6 Y7 Z18 X19'),
    (3, 4): 1.0 * QubitOperator('X9 Z11 Z13 Y14'),
    (3, 5): 1.0 * QubitOperator('Y9 Z11 Z16 X17'),
    (3, 6): 1.0 * QubitOperator('Z9 X10 Z18 Y19'),
    (4, 5): 1.0 * QubitOperator('X12 Z14 Z16 Y17'),
    (4, 6): 1.0 * QubitOperator('Y12 Z14 Z19 X20'),
    (5, 6): 1.0 * QubitOperator('X15 Z17 Z19 Y20'),
    (7, 8): -1.0 * QubitOperator('X21 Z23 Z25 Y26'),
    (7, 9): 1.0 * QubitOperator('Y21 Z23 Z28 X29'),
    (7, 10): 1.0 * QubitOperator('Z21 X22 Z30 Y31'),
    (7, 11): 1.0 * QubitOperator('Z21 Y22 Z33 X34'),
    (7, 12): 1.0 * QubitOperator('Z22 X23 Y36 Z38'),
    (7, 13): 1.0 * QubitOperator('Z22 Y23 X39 Z41'),
    (8, 9): 1.0 * QubitOperator('X24 Z26 Z28 Y29'),
    (8, 10): 1.0 * QubitOperator('Y24 Z26 Z31 X32'),
    (8, 11): 1.0 * QubitOperator('Z24 X25 Z33 Y34'),
    (8, 12): 1.0 * QubitOperator('Z24 Y25 Z36 X37'),
    (8, 13): 1.0 * QubitOperator('Z25 X26 Y39 Z41'),
    (9, 10): 1.0 * QubitOperator('X27 Z29 Z31 Y32'),
    (9, 11): 1.0 * QubitOperator('Y27 Z29 Z34 X35'),
    (9, 12): 1.0 * QubitOperator('Z27 X28 Z36 Y37'),
    (9, 13): 1.0 * QubitOperator('Z27 Y28 Z39 X40'),
    (10, 11): 1.0 * QubitOperator('X30 Z32 Z34 Y35'),
    (10, 12): 1.0 * QubitOperator('Y30 Z32 Z37 X38'),
    (10, 13): 1.0 * QubitOperator('Z30 X31 Z39 Y40'),
    (11, 12): 1.0 * QubitOperator('X33 Z35 Z37 Y38'),
    (11, 13): 1.0 * QubitOperator('Y33 Z35 Z40 X41'),
    (12, 13): 1.0 * QubitOperator('X36 Z38 Z40 Y41'),
}

In [13]:
test_mapping(aops, bops, stabs)

No irregularities found.


In [14]:
from logical_vector_simulator import logical_state_simulation
from logical_vector_construction import reduce_to_logical_plus
from noise import noiseless

In [15]:
ll = logical_state_simulation(
    stabilizers = stabs,
    logical_operators = [bops[x] for x in range(6)] + [bops[7 + x] for x in range(6)],
    n_phys_qubits = 42,
    )

In [16]:
test_mapping(
    aops,
    bops,
    ll[4][0],
    )
for j in range(len(ll[4][0])):
    print(f'original {j}: {stabs[j]}')
    print(f'updated {j}: {ll[4][0][j]}')

No irregularities found.
original 0: (1+0j) [Z0 X3 Z4 X5 Z8]
updated 0: (1+0j) [Z0 X3 Z4 X5 Z8]
original 1: (-1+0j) [Y0 X1 Z2 Y3 Z4 X5 Z9 X10 X11]
updated 1: (-1+0j) [X0 X1 Z2 Z3 Z8 Z9 X10 X11]
original 2: (-1+0j) [Y0 Y1 Z2 Z3 Y4 Y5 Z13]
updated 2: (-1+0j) [Z1 X3 X4 Z5 Z9 X10 X11 Z13]
original 3: (-1+0j) [X0 Z1 Y2 Z3 X4 Y5 X15 X16 Z17]
updated 3: (-1+0j) [X0 Z1 Y2 Z3 X4 Y5 X15 X16 Z17]
original 4: (1+0j) [X0 Z1 X2 Z5 Z18]
updated 4: (-1+0j) [Z2 Z3 X4 X5 X15 X16 Z17 Z18]
original 5: (1+0j) [X0 X1 Z2 X6 Z7 Y8 Z9 X10 Y11]
updated 5: (-1+0j) [Z3 X6 Z7 X8 Z11]
original 6: (-1+0j) [X0 Y1 Z2 Y6 Z7 Y8 Z12 Y13 X14]
updated 6: (1-0j) [Z1 Z6 Z9 X10 Y11 Z12 Y13 X14]
original 7: (-1+0j) [Y0 Z1 Y2 Z6 Y7 X8 X15 Y16 Z17]
updated 7: (1-0j) [Z3 Z4 Y7 Y8 Z11 Z12 X13 X14 Z16]
original 8: (1+0j) [Y0 Z1 X2 Z6 X7 X8 Y18 X19 Z20]
updated 8: (1-0j) [Z2 Z7 X15 Y16 Z17 Y18 X19 Z20]
original 9: (1+0j) [Z1 Y9 Y10 Z11 Z12 Y13 Y14]
updated 9: (-1+0j) [Z6 X9 Z10 X11 Z14]
original 10: (-1+0j) [Z0 Y1 X2 X9 Y10 Z11 Y15 

In [17]:
li = {}
for x in aops.keys():
    li.update({x: reduce_to_logical_plus(aops[x],*ll[4])[0][0]})
aops_new = dict(li)

li = {}
for x in bops.keys():
    li.update({x: reduce_to_logical_plus(bops[x],*ll[4])[0][0]})
bops_new = dict(li)

reduce_to_logical_plus(aops[0,1],*ll[4])[0][0]

-1.0 [Y0]

In [18]:
test_mapping(aops_new, bops_new, [])

No irregularities found.


In [19]:
print([ll[4][0][x] - stabs[x] for x in range(len(stabs))])

[0, (-1+0j) [X0 X1 Z2 Z3 Z8 Z9 X10 X11] +
(1+0j) [Y0 X1 Z2 Y3 Z4 X5 Z9 X10 X11], (1+0j) [Y0 Y1 Z2 Z3 Y4 Y5 Z13] +
(-1+0j) [Z1 X3 X4 Z5 Z9 X10 X11 Z13], 0, (-1+0j) [X0 Z1 X2 Z5 Z18] +
(-1+0j) [Z2 Z3 X4 X5 X15 X16 Z17 Z18], (-1+0j) [X0 X1 Z2 X6 Z7 Y8 Z9 X10 Y11] +
(-1+0j) [Z3 X6 Z7 X8 Z11], (1+0j) [X0 Y1 Z2 Y6 Z7 Y8 Z12 Y13 X14] +
(1-0j) [Z1 Z6 Z9 X10 Y11 Z12 Y13 X14], (1+0j) [Y0 Z1 Y2 Z6 Y7 X8 X15 Y16 Z17] +
(1-0j) [Z3 Z4 Y7 Y8 Z11 Z12 X13 X14 Z16], (-1+0j) [Y0 Z1 X2 Z6 X7 X8 Y18 X19 Z20] +
(1-0j) [Z2 Z7 X15 Y16 Z17 Y18 X19 Z20], (-1+0j) [Z1 Y9 Y10 Z11 Z12 Y13 Y14] +
(-1+0j) [Z6 X9 Z10 X11 Z14], (1+0j) [Z0 Y1 X2 X9 Y10 Z11 Y15 Z16 Y17] +
(1-0j) [Z1 Z4 Z6 X10 Y11 Z13 Z14 Z15 Y16 X17], (-1+0j) [Z0 Y1 Y2 Z10 Y18 Y19 Z20] +
(-1+0j) [Z7 X9 X10 Z11 Z15 X16 X17 Z19], (-1+0j) [Z0 X1 X2 Y12 X13 Z14 Y15 Z16 X17] +
(1-0j) [Z4 Z7 X9 X10 Z11 Y12 Y13 Z14 Z16 Z17 Z19], (-1+0j) [Z0 X1 Y2 X12 X13 Z14 X18 Z19 Y20] +
(-1+0j) [X9 X10 Z11 Z12 Z17 Z18 X19 X20], 0, 0, (-1+0j) [X21 X22 Z23 Z24 Z29 Z30 X31 X32]